# Install and import some modules



In [ ]:
# after installing, need to restart runtime for changes to take effect
# the nbclient error may be ignored for the time being
!pip install web3 > /dev/null

In [ ]:
# 必要なライブラリのインストール
!apt install libzbar0
!pip install pyzbar

In [ ]:
# import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time
import csv
import datetime
import json

In [ ]:
from pyzbar.pyzbar import decode

#サポートパッチのインポート
from google.colab.patches import cv2_imshow
# cv2_imshow(image)

In [ ]:
import time
from web3 import Web3

# You should fill out this network below.
web3 = Web3(Web3.HTTPProvider(""))

In [ ]:
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

In [ ]:
# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;

    var pendingResolve = null;
    var shutdown = false;

    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }

    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }

    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);

      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);

      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);

      const instruction = document.createElement('div');
      instruction.innerHTML =
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };

      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);

      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();

      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }

      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }

      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;

      return {'create': preShow - preCreate,
              'show': preCapture - preShow,
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)

def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

In [ ]:
import math

def contrast(image, a):
    lut = [ np.uint8(255.0 / (1 + math.exp(-a * (i - 128.) / 255.))) for i in range(256)]
    result_image = np.array( [ lut[value] for value in image.flat], dtype=np.uint8 )
    result_image = result_image.reshape(image.shape)
    return result_image

# Trash Can initialaization

In [ ]:
# the contract address and the ABI value set in the next cell need to be changed when
# the Solidity Code is changed and the conract is redeployed
contract_address     = ''
wallet_address = ''  # actual address obfuscated

# the following data should not be made public under any circumstances when you working on the MainNET
# as this can be used to withdraw valuable Ethers from the wallet mentioned above
# here we make it public because the wallet is on a Test Chain (rinkbe) and the ether coins in the wallet have
# have no financial value

wallet_private_key = ''  # actual private_key obfuscated

In [ ]:
#@title ABI value
abi = json.loads('[{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"owner","type":"address"},{"indexed":true,"internalType":"address","name":"spender","type":"address"},{"indexed":false,"internalType":"uint256","name":"value","type":"uint256"}],"name":"Approval","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"worker","type":"address"},{"indexed":true,"internalType":"uint256","name":"id","type":"uint256"}],"name":"DoneWork","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"worker","type":"address"},{"indexed":true,"internalType":"uint256","name":"id","type":"uint256"}],"name":"StartWork","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"from","type":"address"},{"indexed":true,"internalType":"address","name":"to","type":"address"},{"indexed":false,"internalType":"uint256","name":"value","type":"uint256"}],"name":"Transfer","type":"event"},{"inputs":[{"internalType":"address","name":"owner","type":"address"},{"internalType":"address","name":"spender","type":"address"}],"name":"allowance","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"spender","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"approve","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"account","type":"address"}],"name":"balanceOf","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"decimals","outputs":[{"internalType":"uint8","name":"","type":"uint8"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"spender","type":"address"},{"internalType":"uint256","name":"subtractedValue","type":"uint256"}],"name":"decreaseAllowance","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"actualWeight","type":"uint256"},{"internalType":"bytes32","name":"messageHash","type":"bytes32"},{"internalType":"bytes","name":"signature","type":"bytes"}],"name":"doneWork","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"getTrashCans","outputs":[{"components":[{"internalType":"uint256","name":"id","type":"uint256"},{"internalType":"address","name":"trashCanOwner","type":"address"},{"internalType":"address","name":"trashCan","type":"address"},{"internalType":"string","name":"trashCanLocationAddress","type":"string"},{"internalType":"string","name":"trashCanLocationImage","type":"string"},{"internalType":"uint256","name":"trashCanLatitude","type":"uint256"},{"internalType":"uint256","name":"trashCanLongitude","type":"uint256"},{"internalType":"uint256","name":"trashCanAmount","type":"uint256"},{"internalType":"uint256","name":"trashCanMaxAmount","type":"uint256"},{"internalType":"uint256","name":"trashCanReward","type":"uint256"},{"components":[{"internalType":"uint256","name":"id","type":"uint256"},{"internalType":"string","name":"recyclingCenterName","type":"string"},{"internalType":"string","name":"recyclingCenterLocationAddress","type":"string"},{"internalType":"uint256","name":"recyclingCenterLatitude","type":"uint256"},{"internalType":"uint256","name":"recyclingCenterLongitude","type":"uint256"}],"internalType":"struct SALVARE.RecyclingCenter","name":"recyclingCenter","type":"tuple"}],"internalType":"struct SALVARE.TrashCan[]","name":"","type":"tuple[]"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"target","type":"address"}],"name":"getWork","outputs":[{"components":[{"internalType":"uint256","name":"id","type":"uint256"},{"internalType":"uint256","name":"gram","type":"uint256"}],"internalType":"struct SALVARE.Work","name":"","type":"tuple"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"spender","type":"address"},{"internalType":"uint256","name":"addedValue","type":"uint256"}],"name":"increaseAllowance","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"worker","type":"address"}],"name":"isWorking","outputs":[{"components":[{"internalType":"uint256","name":"id","type":"uint256"},{"internalType":"uint256","name":"gram","type":"uint256"}],"internalType":"struct SALVARE.Work","name":"","type":"tuple"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"name","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"","type":"address"}],"name":"requesterBalances","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256","name":"_newAmount","type":"uint256"},{"internalType":"uint256","name":"id","type":"uint256"}],"name":"setAmountOfTrash","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"trashWeight","type":"uint256"},{"internalType":"bytes32","name":"messageHash","type":"bytes32"},{"internalType":"bytes","name":"signature","type":"bytes"},{"internalType":"uint256","name":"id","type":"uint256"}],"name":"startWork","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"symbol","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"totalSupply","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"transfer","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"from","type":"address"},{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"transferFrom","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"bytes32","name":"messageHash","type":"bytes32"},{"internalType":"bytes","name":"signature","type":"bytes"}],"name":"verifySignature","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"pure","type":"function"},{"inputs":[{"internalType":"address","name":"","type":"address"}],"name":"workerToWork","outputs":[{"internalType":"uint256","name":"id","type":"uint256"},{"internalType":"uint256","name":"gram","type":"uint256"}],"stateMutability":"view","type":"function"}]')

In [ ]:
contract = web3.eth.contract(address = contract_address, abi = abi)

# a someone's Wallet Blance

In [ ]:
# you can make sure your total balance
contract.functions.balanceOf("").call()

In [ ]:
contract.functions.balanceOf("").call()

47000000000000000000000

# QR code reader at once

In [ ]:
# start streaming video from webcam
video_stream()
# label for video
label_html = 'Capturing...'
# initialize bounding box to empty
bbox = ''
count = 0
data_pre = []  # 読んだバーコードの履歴
pre_bar = ''

BarcodeResultFile = 'Barcode_reader_result.csv'

dt_now = datetime.datetime.now()
with open(BarcodeResultFile, 'a') as f:
    writer = csv.writer(f, lineterminator='\n')  # 改行コード指定
    writer.writerow(['Barcode start: ' + str(dt_now)])

# Flag variable
barcode_found = False

while True:
    # 処理時間測定用 1 of 2
    start_time = time.time()

    # print('I am before video_frame')
    js_reply = video_frame(label_html, bbox)
    # print('I am after video_frame')
    if not js_reply:
        break

    # convert JS response to OpenCV Image
    img = js_to_image(js_reply["img"])

    # create transparent overlay for bounding box
    bbox_array = np.zeros([480, 640, 4], dtype=np.uint8)

    # GrayScale
    imageGlay = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = contrast(imageGlay, 5)

    data = decode(img)

    # コード内容を出力
    for lc in range(len(data)):
        count = 0
        # Barcodeの枠を描画する
        pts = np.array([data[lc].polygon], np.int32)
        pts = pts.reshape((-1, 1, 2))
        cv2.polylines(bbox_array, [pts], True, (0, 0, 255), 5)

        # Barcodeのデータを処理する
        barcode_data = data[lc][0].decode('utf-8', 'ignore')
        # print(barcode_data, "Barcord Data")

        # Barcodeのデータを描画する
        bbox_array = cv2.putText(bbox_array, barcode_data, (50, 50 + lc * 70), cv2.FONT_HERSHEY_SIMPLEX, 0.8,
                                 (255, 0, 0), 2)

        result = barcode_data not in data_pre
        if result:
            pre_bar = barcode_data
            data_pre.append(barcode_data)

            with open(BarcodeResultFile, 'a') as f:
                writer = csv.writer(f, lineterminator='\n')
                writer.writerow([barcode_data])
                # writer.writerow([[barcode_data]])

            # Set the flag to True and break the loop
            barcode_found = True
            break
        else:
            if barcode_data == pre_bar:
                bbox_array = cv2.putText(bbox_array, 'OK! Bring next barcode', (150, 240),
                                         cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 0, 0), 3)
            else:
                bbox_array = cv2.putText(bbox_array, 'Duplicate. Read already', (200, 80 + lc * 70),
                                         cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 0, 0), 3)

    # 処理時間の表示 2 of 2
    pro_time = time.time() - start_time
    time_lc = ' : processing time: {0:.2f}'.format(pro_time)
    bbox_array = cv2.putText(bbox_array, str(count) + time_lc, (50, 430), cv2.FONT_HERSHEY_SIMPLEX, 0.8,
                             (255, 0, 0), 2)
    count += 1

    bbox_array[:, :, 3] = (bbox_array.max(axis=2) > 0).astype(int) * 255
    # convert overlay of bbox into bytes
    bbox_bytes = bbox_to_bytes(bbox_array)

    # update bbox so next frame gets new overlay
    bbox = bbox_bytes

    # Check the flag to exit the loop
    if barcode_found:
        break


In [ ]:
parsed_data = json.loads(barcode_data)
trashCan_id = parsed_data["id"]
message_hash = parsed_data["messageHash"]
signature = parsed_data["signature"]

print("trashCan_id =", trashCan_id)
print("messageHash =", message_hash)
print("signature =", signature)

# How much do you want to carry trash?

In [ ]:
trash_amount = 7000

# StartWork

In [ ]:
# Manually build and sign a transaction:
unsent_contract_tx = contract.functions.startWork( trash_amount, message_hash, signature, int(trashCan_id)).build_transaction({
    "from": wallet_address,
    "nonce": web3.eth.get_transaction_count(wallet_address),
    'gas': 400000,
    'gasPrice': web3.to_wei('40', 'gwei'),
})
signed_tx = web3.eth.account.sign_transaction(unsent_contract_tx, private_key=wallet_private_key)

tx_hash = web3.eth.send_raw_transaction(signed_tx.rawTransaction)
web3.eth.wait_for_transaction_receipt(tx_hash)

# QR code reader at once

In [ ]:
# start streaming video from webcam
video_stream()
# label for video
label_html = 'Capturing...'
# initialize bounding box to empty
bbox = ''
count = 0
data_pre = []  # 読んだバーコードの履歴
pre_bar = ''

BarcodeResultFile = 'Barcode_reader_result.csv'

dt_now = datetime.datetime.now()
with open(BarcodeResultFile, 'a') as f:
    writer = csv.writer(f, lineterminator='\n')  # 改行コード指定
    writer.writerow(['Barcode start: ' + str(dt_now)])

# Flag variable
barcode_found = False

while True:
    # 処理時間測定用 1 of 2
    start_time = time.time()

    # print('I am before video_frame')
    js_reply = video_frame(label_html, bbox)
    # print('I am after video_frame')
    if not js_reply:
        break

    # convert JS response to OpenCV Image
    img = js_to_image(js_reply["img"])

    # create transparent overlay for bounding box
    bbox_array = np.zeros([480, 640, 4], dtype=np.uint8)

    # GrayScale
    imageGlay = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = contrast(imageGlay, 5)

    data = decode(img)

    # コード内容を出力
    for lc in range(len(data)):
        count = 0
        # Barcodeの枠を描画する
        pts = np.array([data[lc].polygon], np.int32)
        pts = pts.reshape((-1, 1, 2))
        cv2.polylines(bbox_array, [pts], True, (0, 0, 255), 5)

        # Barcodeのデータを処理する
        barcode_data = data[lc][0].decode('utf-8', 'ignore')
        # print(barcode_data, "Barcord Data")

        # Barcodeのデータを描画する
        bbox_array = cv2.putText(bbox_array, barcode_data, (50, 50 + lc * 70), cv2.FONT_HERSHEY_SIMPLEX, 0.8,
                                 (255, 0, 0), 2)

        result = barcode_data not in data_pre
        if result:
            pre_bar = barcode_data
            data_pre.append(barcode_data)

            with open(BarcodeResultFile, 'a') as f:
                writer = csv.writer(f, lineterminator='\n')
                writer.writerow([barcode_data])
                # writer.writerow([[barcode_data]])

            # Set the flag to True and break the loop
            barcode_found = True
            break
        else:
            if barcode_data == pre_bar:
                bbox_array = cv2.putText(bbox_array, 'OK! Bring next barcode', (150, 240),
                                         cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 0, 0), 3)
            else:
                bbox_array = cv2.putText(bbox_array, 'Duplicate. Read already', (200, 80 + lc * 70),
                                         cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 0, 0), 3)

    # 処理時間の表示 2 of 2
    pro_time = time.time() - start_time
    time_lc = ' : processing time: {0:.2f}'.format(pro_time)
    bbox_array = cv2.putText(bbox_array, str(count) + time_lc, (50, 430), cv2.FONT_HERSHEY_SIMPLEX, 0.8,
                             (255, 0, 0), 2)
    count += 1

    bbox_array[:, :, 3] = (bbox_array.max(axis=2) > 0).astype(int) * 255
    # convert overlay of bbox into bytes
    bbox_bytes = bbox_to_bytes(bbox_array)

    # update bbox so next frame gets new overlay
    bbox = bbox_bytes

    # Check the flag to exit the loop
    if barcode_found:
        break


In [ ]:
parsed_data = json.loads(barcode_data)
trashCan_id = parsed_data["id"]
message_hash = parsed_data["messageHash"]
signature = parsed_data["signature"]

print("trashCan_id =", trashCan_id)
print("messageHash =", message_hash)
print("signature =", signature)

# DoneWork

In [ ]:
# Manually build and sign a transaction:
unsent_contract_tx = contract.functions.doneWork(trash_amount, message_hash, signature).build_transaction({
    "from": wallet_address,
    "nonce": web3.eth.get_transaction_count(wallet_address),
    'gas': 400000,
    'gasPrice': web3.to_wei('40', 'gwei'),
})
signed_tx = web3.eth.account.sign_transaction(unsent_contract_tx, private_key=wallet_private_key)

tx_hash = web3.eth.send_raw_transaction(signed_tx.rawTransaction)
web3.eth.wait_for_transaction_receipt(tx_hash)


# setAmountOfTrash


In [ ]:
# Reference the deployed contract:

# Manually build and sign a transaction:
unsent_contract_tx = contract.functions.setAmountOfTrash(10000, 5).build_transaction({
    "from": wallet_address,
    "nonce": web3.eth.get_transaction_count(wallet_address),
    'gas': 400000,
    'gasPrice': web3.to_wei('40', 'gwei'),
})
signed_tx = web3.eth.account.sign_transaction(unsent_contract_tx, private_key=wallet_private_key)
tx_hash = web3.eth.send_raw_transaction(signed_tx.rawTransaction)
web3.eth.wait_for_transaction_receipt(tx_hash)

In [ ]:
# Send the raw transaction:
# assert billboard.functions.message().call() == "gm"
# tx_hash = web3.eth.send_raw_transaction(signed_tx.rawTransaction)
# web3.eth.wait_for_transaction_receipt(tx_hash)
# assert billboard.functions.message().call() == "gn"

# QR code reader(non stop)

In [ ]:
# start streaming video from webcam
video_stream()
# label for video
label_html = 'Capturing...'
# initialze bounding box to empty
bbox = ''
count = 0
data_pre =[] # 読んだバーコードの履歴
pre_bar = ''

BarcodeResultFile = 'Barcode_reader_result.csv'

dt_now = datetime.datetime.now()
with open(BarcodeResultFile, 'a') as f:
    writer = csv.writer(f, lineterminator='\n') # 改行コード指定
    writer.writerow(['Barcode start : '+str(dt_now)])

while True:
    # 処理時間測定用 1 of 2
    start_time = time.time()

    # print('I am before video_frame')
    js_reply = video_frame(label_html, bbox)
    # print('I am after video_frame')
    if not js_reply:
        break

    # convert JS response to OpenCV Image
    img = js_to_image(js_reply["img"])

    # create transparent overlay for bounding box
    bbox_array = np.zeros([480,640,4], dtype=np.uint8)

    # GrayScale
    imageGlay = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = contrast(imageGlay, 5)

    data = decode(img)

    # コード内容を出力
    for lc in range(len(data)):
        count = 0
        # Barcodeの枠を描画する
        pts = np.array([data[lc].polygon],np.int32)
        pts = pts.reshape((-1,1,2))
        cv2.polylines(bbox_array,[pts],True,(0,0,255),5)

        # Barcodeのデータを処理する
        barcode_data = data[lc][0].decode('utf-8', 'ignore')
        # print(barcode_data, "Barcord Data")

        # Barcodeのデータを描画する
        bbox_array = cv2.putText(bbox_array,barcode_data,(50,50+lc*70),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,0,0),2)

        result = barcode_data not in data_pre
        if ( result ):
            pre_bar = barcode_data
            data_pre.append(barcode_data)

            with open(BarcodeResultFile, 'a') as f:
                writer = csv.writer(f, lineterminator='\n')
                writer.writerow([barcode_data])
                # writer.writerow([[barcode_data]])

        else:
            if barcode_data == pre_bar:
                bbox_array = cv2.putText(bbox_array,'OK! Bring next barcode',(150,240),cv2.FONT_HERSHEY_SIMPLEX,1.0,(255,0,0),3)

            else:
                bbox_array = cv2.putText(bbox_array,'Duplicate. Read alredy',(200,80+lc*70),cv2.FONT_HERSHEY_SIMPLEX,1.0,(255,0,0),3)

    # 処理時間の表示 2 of 2
    pro_time = time.time() - start_time
    time_lc = ' : processing time: {0:.2f}'.format(pro_time)
    bbox_array = cv2.putText(bbox_array,str(count)+time_lc,(50,430),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,0,0),2)
    count += 1

    bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
    # convert overlay of bbox into bytes
    bbox_bytes = bbox_to_bytes(bbox_array)

    # update bbox so next frame gets new overlay
    bbox = bbox_bytes



In [ ]:
parsed_data = json.loads(barcode_data)
trashCan_id = parsed_data["id"]
message_hash = parsed_data["messageHash"]
signature = parsed_data["signature"]

print("trashCan_id =", trashCan_id)
print("messageHash =", message_hash)
print("signature =", signature)